In [3]:
!pip install requests
!pip install beautifulsoup4
!pip3 install imdbpy wikipedia
import wikipedia
import requests
import pandas

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
movies = pandas.read_csv("inputs/movies.csv")
sample = pandas.read_csv("inputs/sample_ratings.csv", index_col=0)

In [5]:
def retrieve_wiki_content(title):
    keywords = []
    page = None
    film_page = None
    try:
        page = wikipedia.page(title)
    except:
        pass
  
    try:
        film_page = wikipedia.page(title + ' (film)')
    except:
        pass

    if page:
        keywords.extend(page.links)
        keywords.extend(page.categories)
    if film_page:
        keywords.extend(film_page.links)
        keywords.extend(film_page.categories)

    keywords = list(set(keywords))
    if len(keywords) > 0:
        keywords = [item.replace(' ','') for item in keywords]
        return keywords
    return None

In [7]:
unique_movie_IDs = sample['movieId'].unique()
all_terms = []
all_movie_terms = {}
counter = 1
for movieID in unique_movie_IDs:
    counter += 1
    t = movies[movies['movieId']==movieID]['title'].iloc[0]
    t = str.strip(t)
    orig_title = t
    year = None
    try:
        orig_title = t[:-7]
        year = int(t[-5:-1])
    except:
        pass
    wiki = retrieve_wiki_content(orig_title)
    if wiki:
        all_terms.extend(wiki)
    all_movie_terms[movieID] = (orig_title, wiki)

KeyboardInterrupt: 

In [ ]:
# manually examine links & categories tags and remove ones that are too rare and ones that are too common
from collections import Counter
ordered_by_commonness = Counter(all_terms).most_common()
ordered_by_commonness

[('IMDb', 961),
 ('RottenTomatoes', 940),
 ('English-languagefilms', 875),
 ('Americanfilms', 803),
 ('BoxOfficeMojo', 797),
 ('Articleswithshortdescription', 710),
 ('Metacritic', 629),
 ('AllMovie', 586),
 ('Templatefilmdatewith1releasedate', 549),
 ('RogerEbert', 512),
 ('ISBN(identifier)', 499),
 ('TheNewYorkTimes', 492),
 ('ShortdescriptionmatchesWikidata', 451),
 ('Variety(magazine)', 449),
 ('BritishBoardofFilmClassification', 341),
 ('ChicagoSun-Times', 340),
 ('ShortdescriptionisdifferentfromWikidata', 332),
 ('LosAngelesTimes', 331),
 ('Allarticleswithunsourcedstatements', 314),
 ('Templatefilmdatewith2releasedates', 305),
 ('EntertainmentWeekly', 301),
 ('CinemaScore', 294),
 ('TurnerClassicMovies', 265),
 ('TheHollywoodReporter', 262),
 ('WorldCatIdentities(identifier)', 257),
 ('VIAF(identifier)', 242),
 ('WikipediaarticleswithVIAFidentifiers', 242),
 ('WaybackMachine', 240),
 ('DVD', 240),
 ('WikipediaarticleswithGNDidentifiers', 239),
 ('GND(identifier)', 239),
 ('Webarc

Here, We see the highest frequency term is IMDB with frequency 964 / 1000 movies (means it's probably not that useful), same with Rotten Tomatoes (941/1000 movies). Let us remove some specific high-freq terms as well as some very low frequency terms (anything that occurs for less than 20 unique movies out of 1000).

In [ ]:
terms_to_remove = ['IMDb', 'RottenTomatoes', 'English-languagefilms', 'Americanfilms', 'BoxOfficeMojo', 'Articleswithshortdescription', 'Metacritic', 'AllMovie', 'RogerEbert', 'ShortdescriptionmatchesWikidata', 'ISBN(identifier)']
terms_to_remove.extend([x[0] for x in ordered_by_commonness if 20 > x[1]])

In [ ]:
final_impt_terms = list(set(all_terms) - set(terms_to_remove))
print(len(final_impt_terms))

1077


We end up with 1077 final good terms from Wikipedia that we'd like to keep track of! So now, let's make a dataframe associates each movie_id with its title and terms.


In [ ]:
for movie_id, tup in all_movie_terms.items()
    if tup[1]:
        updated_list = [x for x in tup[1] if x in final_impt_terms]
        all_movie_terms[movie_id] = (tup[0], ' '.join(updated_list))


In [ ]:
tags = pandas.read_csv('inputs/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [ ]:
movies_to_include = list(sample['movieId'].unique())
final_tags = tags[tags['movieId'].isin(movies_to_include)]
final_tags.tag = final_tags.tag.str.replace(' ', '')
agg_movie_tags = final_tags.groupby('movieId')['tag'].agg(lambda x: list(x))
agg_movie_tags

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


movieId
2         [fantasy, adaptedfrom:book, animals, badcgi, b...
5         [stevemartin, stevemartin, pregnancy, remake, ...
21        [funny!, GeneHackman, Hilarious!, hollywood, J...
26        [Shakespeare, great, Nudity(Topless), Shakespe...
35        [entirelydialogue, bisexual, characterdriven, ...
                                ...                        
174055    [Boring, ChristopherNolan, history, Writer:Chr...
177765    [afterlife, animation, colorful, death, Diadel...
180045    [plane, toplist17, AaronSorkin, crime, biograp...
182823    [fantasy, orcs, Willsmith, action, cops, fanta...
187541    [feminism, jackjack, predictable, superhero, a...
Name: tag, Length: 1000, dtype: object

In [ ]:
all_tags = []
for tag_list in agg_movie_tags:
    all_tags.extend(list(set(tag_list)))
tag_count = Counter(all_tags).most_common()

After inspecting the counter element, we saw here that the most common tag is CLV which appears for 179 / 1000 movies. Thus, we don't need to remove any super high frequency tags since none occur for most every movie. We do have some tags that occur very few times. Based on manual inspection, we remove tags that occur less than 5 times. We also remove some manually found garbage tags below.

In [ ]:
tags_to_remove = [tag[0] for tag in tag_count if not any(char.isalpha() or char.isdigit() for char in tag[0])] # all punctuation tags
tags_to_remove.extend([tag[0] for tag in tag_count if tag[1] < 5])

In [ ]:
for movie_id, lst in zip(agg_movie_tags.index, agg_movie_tags):
    if lst:
        updated_list = [x for x in lst if x not in tags_to_remove]
        agg_movie_tags.loc[movie_id] = updated_list

In [ ]:
agg_movie_tags

movieId
2         [fantasy, adaptedfrom:book, animals, basedonab...
5         [pregnancy, remake, aging, baby, daughter, mid...
21        [hollywood, JohnTravolta, mafia, satire, airpo...
26        [great, Nudity(Topless), friends, jealousy, sh...
35        [bisexual, homosexuality, intense, polyamory, ...
                                ...                        
174055    [Boring, history, Boring, historical, history,...
177765    [afterlife, animation, colorful, death, disney...
180045    [crime, biographical, smart, biographical, cou...
182823    [fantasy, action, fantasy, losangeles, magic, ...
187541    [feminism, predictable, superhero, animated, C...
Name: tag, Length: 1000, dtype: object

In [ ]:
final_term_dict = {}
for movie_id, tup in all_movie_terms.items():
    if tup[1]:
        tag_str = ' '.join(agg_movie_tags.loc[movie_id])
        extended_list = tup[1] + ' ' + tag_str
        final_term_dict[movie_id] = (tup[0], extended_list)
    else:
        final_term_dict[movie_id] = tup

In [ ]:
final_series = pandas.Series([final_term_dict[movie_id][1] for movie_id in list(final_term_dict.keys())], index=list(final_term_dict.keys()))
final_series.fillna("",inplace=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(final_series)
movie_vector_df = pandas.DataFrame.sparse.from_spmatrix(count_matrix, columns=count.get_feature_names(), index=final_series.index)


['100cheers', '100essentialfemaleperformances', '100greatestmovies', '100heroes', '100heroesandvillains', '100laughs', '100moviequotes', '100movies', '100passions', '100thrills', '10thanniversaryedition', '12yearsaslave', '13', '18thcentury', '1917', '1930s', '1937film', '1940s', '1948film', '1949film', '1950film', '1950s', '1956film', '1958film', '1959film', '1960s', '1961film', '1963film', '1964film', '1966film', '1968film', '1969film', '1970s', '1972film', '1973film', '1977film', '1978film', '1979film', '1980film', '1980s', '1985film', '1986tvprogram', '1989film', '1989films', '1990film', '1990s', '1991film', '1992disneyfilm', '1992film', '1993films', '1994films', '1995film', '1995films', '1996film', '1996films', '1997film', '1997films', '1998film', '1998films', '1999film', '1999films', '19pandemic', '19thcentury', '2000film', '2000films', '2001', '2001films', '2002film', '2002films', '2003films', '2004film', '2004films', '2005film', '2005films', '2006film', '2006films', '2007film',

In [ ]:
movie_vector_df.to_csv("inputs/movie_tag_wiki_vecs.csv")